In [3]:
using JuMP, Pkg
using CSV, DataFrames, Statistics
Pkg.add("Ipopt")

returndf = CSV.read("RA Sample Model Data.csv", DataFrame)
returndf

    Updating registry at `~/.julia/registries/General.toml`
   Resolving package versions...
   Installed Ipopt ───── v1.6.2
   Installed Ipopt_jll ─ v300.1400.1400+0
   Installed SPRAL_jll ─ v2024.1.18+0
   Installed Hwloc_jll ─ v2.10.0+0
   Installed ASL_jll ─── v0.1.3+0
    Updating `~/.julia/environments/v1.10/Project.toml`
  [b6b21f68] + Ipopt v1.6.2
    Updating `~/.julia/environments/v1.10/Manifest.toml`
  [b6b21f68] + Ipopt v1.6.2
  [ae81ac8f] + ASL_jll v0.1.3+0
  [e33a78d0] + Hwloc_jll v2.10.0+0
  [9cc047cb] + Ipopt_jll v300.1400.1400+0
  [319450e9] + SPRAL_jll v2024.1.18+0
Precompiling project...
  ✓ ASL_jll
  ✓ Hwloc_jll
  ✓ SPRAL_jll
  ✓ Ipopt_jll
  ✓ Ipopt
  5 dependencies successfully precompiled in 20 seconds. 74 already precompiled.


Row,Date,S&P 500,Bitcoin,Emerging Market Index
,String7,Float64,Float64,Float64
1,04/2024,-0.00923806,-0.0695962,0.0064513
2,03/2024,0.0310188,0.144183,0.0218034
3,02/2024,0.0517206,0.459912,0.0462595
4,01/2024,0.0158957,-0.000423161,-0.0468283
5,12/2023,0.0442292,0.126569,0.0371188
6,11/2023,0.0891793,0.0930091,0.0785621
7,10/2023,-0.0219797,0.277977,-0.0394425
8,09/2023,-0.0487193,0.0332556,-0.0281028
9,08/2023,-0.0177164,-0.102231,-0.0635967


In [48]:
bitcoin_mean = mean(returndf.Bitcoin)
sp500_mean = mean(returndf[!, Symbol("S&P 500")])
em_mean = mean(returndf[!, Symbol("Emerging Market Index")])
print(string(bitcoin_mean) * " ")
print(string(sp500_mean) * " " * string(em_mean))


0.08050057183333333 0.01939490325 0.006966623166666664

In [38]:
em_returns = vec(returndf[!, Symbol("Emerging Market Index")])
sp500_returns = vec(returndf[!, Symbol("S&P 500")])
bitcoin_returns = vec(returndf.Bitcoin)
#Mean return vector
mean_returns = [mean(em_returns),mean(sp500_returns),mean(bitcoin_returns)]
display(mean_returns)
#Cov Matrix    
var_matrix = hcat(em_returns, sp500_returns, bitcoin_returns)

print(string(var(em_returns)) * "\n" * string(var(sp500_returns)) * "\n" * string(var(sp500_returns)))
cov_matrix = cov(var_matrix)


3-element Vector{Float64}:
 0.006966623166666664
 0.01939490325
 0.08050057183333333

0.002088893978125414
0.001609673499168793
0.001609673499168793

3×3 Matrix{Float64}:
 0.00208889  0.0015004   0.00245769
 0.0015004   0.00160967  0.00225019
 0.00245769  0.00225019  0.0269793

In [47]:
#function MeanVarianceOptProblemV1(em, sp500, btc, covariance)    
using Ipopt
#x = [1,1,1]
#x
model = Model(Ipopt.Optimizer)
@variable(model, x[1:3])
@objective(model, Min, transpose(x)*cov_matrix*x)
@constraint(model, weightUL, x[1:3].<=1)
@constraint(model, weightLL, x[1:3].>=0)
@constraint(model, weightAdd, sum(x)<=1)
@constraint(model, returnRequired, transpose(mean_returns)*x >= 0.02)

#print(model)

optimize!(model)

#is_solved_and_feasible(model)
#solution_summary(model)
objective_value(model)

This is Ipopt version 3.14.14, running with linear solver MUMPS 5.6.2.

Number of nonzeros in equality constraint Jacobian...:        0
Number of nonzeros in inequality constraint Jacobian.:       12
Number of nonzeros in Lagrangian Hessian.............:        6

Total number of variables............................:        3
                     variables with only lower bounds:        0
                variables with lower and upper bounds:        0
                     variables with only upper bounds:        0
Total number of equality constraints.................:        0
Total number of inequality constraints...............:        8
        inequality constraints with only lower bounds:        4
   inequality constraints with lower and upper bounds:        0
        inequality constraints with only upper bounds:        4

iter    objective    inf_pr   inf_du lg(mu)  ||d||  lg(rg) alpha_du alpha_pr  ls
   0  0.0000000e+00 2.00e-02 4.82e-01  -1.0 0.00e+00    -  0.00e+00 0.00e+00 

0.0011322529730415306